# Quantitative sensitivity analysis


## Generalized Sobol Indices


Here we show how to compute generalized Sobol indices on the **EOQ** model using the algorithm presented in Kucherenko et al. 2012. We import our model function from ``temfpy`` and use the Kucherenko indices function from ``econsa``. 

In [1]:
import matplotlib.pyplot as plt  # noqa: F401
import numpy as np

from temfpy.uncertainty_quantification import eoq_model

# TODO: Reactivate once Tim's PR is ready.
# from econsa.kucherenko import kucherenko_indices  # noqa: E265

The function ``kucherenko_indices`` expects the input function to be broadcastable over rows, that is, a row represents the input arguments for one evaluation. For sampling around the mean parameters we specify a diagonal covariance matrix, where the variances depend on the scaling of the mean. Since the variances of the parameters are unknown prior to our analysis we choose values such that the probability of sampling negative values is negligible. We do this since the **EOQ** model is not defined for negative parameters and the normal sampling does not naturally account for bounds.

In [2]:
def eoq_model_transposed(x):
    """EOQ Model but with variables stored in columns."""
    return eoq_model(x.T)


mean = np.array([1230, 0.0135, 2.15])
cov = np.diag([1, 0.000001, 0.01])

# indices = kucherenko_indices( # noqa: E265
#    func=eoq_model_transposed, # noqa: E265
#    sampling_mean=mean,  # noqa: E265
#    sampling_cov=cov,  # noqa: E265
#    n_draws=1_000_000,  # noqa: E265
#    sampling_scheme="sobol",  # noqa: E265
# )  # noqa: E265

Now we are ready to inspect the results.

In [3]:
# sobol_first = indices.loc[(slice(None), "first_order"), "value"].values  # noqa: E265
# sobol_total = indices.loc[(slice(None), "total"), "value"].values # noqa: E265

# x = np.arange(3)  # the label locations  # noqa: E265
# width = 0.35  # the width of the bars  # noqa: E265

# fig, ax = plt.subplots()  # noqa: E265
# rects1 = ax.bar(x - width / 2, sobol_first, width, label="First-order")  # noqa: E265
# rects2 = ax.bar(x + width / 2, sobol_total, width, label="Total")  # noqa: E265

# ax.set_ylim([0, 1])  # noqa: E265
# ax.legend()  # noqa: E265

# ax.set_xticks(x)  # noqa: E265
# ax.set_xticklabels(["$x_0$", "$x_1$", "$x_2$"])  # noqa: E265
# ax.legend();  # noqa: E265

In [4]:
# fig  # noqa: E265

## Shapley value

In tutorial, we give a brief notational overview of variance-based sensitivity analysis as well as the Shapley value's theoratical framework (:cite:`Song.2016`). Furthermore, we illustrate how to estimate Shapley effects in the context of a model with dependent inputs.

In this illustrative overview, we follow the framework on variance-based sensitivity analysis and Shapley values developed by :cite:`Song.2016`. 

### Variance Based Sensitivity Analysis: A Snapshot

Consider a model with $k$ inputs denoted by $X_K = \{X_1, X_2, X_3, \dots, X_k \}$ where $K = \{1, 2, \dots, k\}$. Consider also $X_J$, which indicates the vector of inputs included in the index set $J \subseteq X$. The uncertainty in $X_K$ is represented by the joint cumulative distribution $G_K$. Furthermore, we denote the joint distribution of inputs included in the index set $J$ as $G_J$ and the marginal distribution of each $X_i$ as $G_i$. The model is treated as a blackbox, and only the model response is analysed. The model response $Y$ is a function of the inputs, i.e., $Y = f(X_K)$ and therefore $f(X_K)$ is stochastic due to the uncertainty in $X_K$ although $f(\cdot)$ is deterministic. Often, $f(\cdot)$ has a complex structure, and does not have a closed form expression. The overall uncertainty in the model output $Y$ caused by $X_K$ is $Var[Y]$, where the variance is calculated with respect to the joint distribution $G_K$. The Shapley value then, helps us to quantify how much of $Var[Y]$ can be attributed to each each $X_i$.

### Shapley Values: A Theoratical Framework

An analogous framework to the one developed for variance-based sensitivity analysis above is apparent in the specification of the Shapley value. Formally, a \textit{k-player game} with the set of players $K = \{1,2, \dots, k\}$ is defined as a real valued function that maps a subset of $K$ to its corresponding cost (or value), i.e., $c: 2^K \rightarrow  {\rm I\!R}$ with $c(\emptyset) = 0$. With this in mind, $c(J)$ then, represents the cost that arises when the players in the subset $J$ of $K$ participate in the game. The Shapley value for player $i$ with respect to $c(\cdot)$ is defined as 

\begin{equation}
\label{eqn:shapley}
v_i = \sum_{J \subseteq K \backslash \{i\}}^{} \frac{(k -|J| - 1)! |J|!}{k!} \cdot (c(J \cup \{i\}) -c(J)),
\end{equation}
where $|J|$ indicates the size of $J$. In other words, $v_i$ is the incremental cost of including player $i$ in set $J$ averaged over all sets $J \subseteq K \backslash \{i\}$.  The Shapley value gives equal weight to each $k$ subset sizes and equal weights amongst the subsets of the same size, which is important in determining the fairness of the variance allocation in the calculation of Shapley effects in variance-based sensitivity analysis (:cite:`Song.2016`).  Reconciling the two frameworks by direct comparison, we can think of the set of $K$ players as the set of inputs of $f(\cdot)$ and define $c(\cdot)$ so that for $J \subseteq K$, $c(J)$ measures the variance of $Y$ caused by the uncertainty of the inputs in $J$. 

The ideal $c(\cdot)$ should satisfy the conditions: $c(\emptyset) = 0$ and $c(K) = Var[Y]$. Two such candidates for such $c(\cdot)$ can be considered, and have been shown to be equivalent are equivalent (:cite:`Song.2016`).
The first cost function is 
\begin{equation}
\label{eqn:costone}
\tilde{c}(J) = Var[E[Y|X_J]].
\end{equation}
This cost function satisfies the two conditions from above and was originally put forth by :cite:`Owen.2014` and later adopted by :cite:`Song.2016` in their paper. The cost function can be rewritten as $\tilde{c}(J) = Var[Y] - E[Var[Y|X_J]]$, and interpreted as the expected reduction in the output variance when the values of $X_J$ are known. The second cost function that satisfies the required conditions is
\begin{equation}
\label{eqn:costtwo}
c(J) = E[Var[Y|X_{-J}]]
\end{equation}
where $X_{-J} = X_{K \backslash J} $. $c(J)$ is interpreted as the expected remaining variance in $Y$ when the values of $X_{-J} $ are known. In this case, the incremental cost $c(J \cup \{i\}) -c(J)$ can be interpreted as the expected decrease in the variance of $Y$ conditional on the known input values of $X_i$ out of all the unknown inputs in $J \cup \{i\}$. 

Although both cost functions result in the same Shapley values, their resultant estimators from Monte Carlo simulation are different. :cite:`Sun.2011` reveal that the Monte Carlo estimator that results from the simulation of $\tilde{c}(J)$ can be severely biased if the inner level sample size used to estimate the conditional expectation is not large enough. Given the already computationally demanding structure of microeconomic models, this added computational complexity is costly. In contrast however, the estimator of $c(J)$ is unbiased for all sample sizes. Because of this added feature, we follow :cite:`Song.2016` in using the cost function $c(J)$ rather that $\tilde{c}(J)$. We therefore define the \textit{Shapley effect} of the $i_{th}$ input, $Sh_i$, as the Shapley value obtained by applying the cost function $c(J)$ to \autoref{eqn:shapley}. Indeed, any Shapley value defined by the satisfaction of the two conditions: $c(\emptyset) = 0$ and $c(K) = Var[Y]$ imply that
\begin{equation}
\label{eqn:variance_sum}
\sum_{k}^{i=1} Sh_i = Var[Y],
\end{equation}
even if there is dependence or structural interactions amongst the elements in $X_K$. 

From here on out, we use $Sh_i$ to denote the Shapley effect and $v_i$ to denote the generic Shapley value.

### Tutorial: A Linear Model with Three Inputs

To get an understanding of just how the Shapley effects can be applied to economic models, we illustrate the method in a simple to follow and understand linear model with three inputs. 

#### The model
Consider the linear model:

\begin{equation}
\label{eq: gaussianlinear}
Y = \beta_{0} + \beta^T \textbf{X}.
\end{equation}

with constants $\beta_{0} \in {\rm I\!R}$, $\beta \in {\rm I\!R^3}$ and $ X \sim \mathcal{N}(0,\,\sum)\,$ with the covariance matrix:

\begin{equation*}
\sum = 
\begin{pmatrix}
\sigma_1^2 & \sigma_{12} & \sigma_{13} \\
 \sigma_{21} & \sigma_2^2 & \sigma_{23}\\
 \sigma_{31} & \sigma_{32} & \sigma_3^2 \\
\end{pmatrix}
\end{equation*}

where $ \sigma_1 > 0, \sigma_2 > 0, \sigma_3 > 0.$

In this simplistic illustration, we have that $\sigma_1 = \sigma_2 = 1$, $\sigma_3 = 4$ and then correlate $X_2$ and $X_3$. The resulting covariance matrix is

\begin{equation*}
\sum = 
\begin{pmatrix}
1.0 & 0 & 0 \\
0 & 1.0 & 1.8\\
0 & 1.8 & 4 \\
\end{pmatrix}
.
\end{equation*}

All eigenvalues of $\sigma$ are positive which verifies that the constructed covariance matrix is positive definite. With this setup, we conduct SA using the Shapley effects method. To use the `get_shapley` function, you will need both the vector of paramters and the covariance matrix.

#### The Execution

In [1]:
# import necessary packages and functions
import numpy as np
import chaospy as cp

from econsa.shapley import get_shapley
from econsa.shapley import _r_condmvn

Load all neccesary inputs for the model, you will need:
- a vector of mean estimates
- a covariance matrix
- the model you are conducting SA on
- the functions ``x_all`` and ``x_cond`` for conditional sampling. These functions depend on the distribution from which you are sampling from - for the purposes of this ilustration, we will sample from a multvariate normal distribution, but the functions can be tailored to needs.

In [4]:
# mean and covaraince matrix inputs
np.random.seed(123)
n_inputs = 3
mean = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])

In [7]:
# model for which SA is being performed
def gaussian_model(X):
    return np.sum(X,1)

In [8]:
# functions for conditional sampling
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis = 1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(n, mean = mean, cov = cov, dependent_ind = subset_j, given_ind = subsetj_conditional, x_given = xjc)

In [5]:
# estimate Shapley effects using the exact method
method = 'exact'
n_perms = None
n_output = 10**4
n_outer = 10**3
n_inner = 10**2

get_shapley(method, gaussian_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

,Shapley effects,std. errors,CI_min,CI_max
X1,0.101309,0.002415,0.096575,0.106044
X2,0.418989,0.162970,0.099568,0.738410
X3,0.479701,0.163071,0.160083,0.799320


In [6]:
# estimate Shapley effects using the random method
method = 'random'
n_perms = 30000
n_output = 10**4
n_outer = 1
n_inner = 3


get_shapley(method, gaussian_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

,Shapley effects,std. errors,CI_min,CI_max
X1,0.111083,0.002969,0.105263,0.116903
X2,0.415180,0.003141,0.409023,0.421337
X3,0.473737,0.003160,0.467543,0.479931


And there we have the resultant Shapley effects, in line witht he chosen method of estimation.